# Young-Cheline Data Analyze

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
import random
import pymysql
import datetime

In [3]:
# Mysql에서 데이터 가져오는 함수
def get_mysql_data(str):
    conn = pymysql.connect(host = "localhost",
             user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute(str)
    data = cursor.fetchall()
    return data

In [4]:
# 부여된 가중치를 활용하여 표본을 n번 추출
def Random_Weighted_Sampling(menu_df,user_df,n):
    prs_data = []
    for i in range(len(menu_df)):
        prs_data.append(menu_df.iloc[i,:].tolist() + random.choices(range(9,15), k=2) + random.choices(range(5,11), k=2) + random.choices(range(1,7), k=1))
    df = pd.DataFrame(columns = ['menu_id','menu_name','restaurant_id','url','1','2','3','4','5'], data = prs_data)
    df_n = len(df)
    user_n = len(user_df)
    id_data = [0 for i in range(user_n)]
    user_df['count']=id_data
    rws_data = []
    #db연결
    conn = pymysql.connect(host = "localhost",
    user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    #계산
    for i in range(n):
        # 무작위로 메뉴 선택
        random_index = random.randrange(df_n)
        user_index = random.randrange(user_n)
        # 메뉴의 정보
        menu_id = df.iloc[random_index].loc['menu_id'].tolist()
        time = datetime.datetime.now()
        user_name = user_df.iloc[user_index].loc['user_name']
        number = i+1
        restaurant_id = df.iloc[random_index].loc['restaurant_id']
        url = df.iloc[random_index].loc['url']
        # 메뉴의 가중치
        weight2 = df.iloc[random_index,[4,8]].to_numpy()
        weightT = df.iloc[random_index,[4,5,6,8]].to_numpy()
        weight4 = df.iloc[random_index,[4,6,7,8]].to_numpy()
        weight5 = df.iloc[random_index,[4,5,6,7,8]].to_numpy()
        # 가중치에 따른 확률 계산
        percent2 = list(weight2/np.sum(weight2))
        percentT = list(weightT/np.sum(weightT))
        percent4 = list(weight4/np.sum(weight4))
        percent5 = list(weight5/np.sum(weight5))
        # 확률에 맞게 추출
        flavor = np.random.choice([1,2,3,4],p=percentT)
        cleaning = np.random.choice([None,1,2,3],p=percent4)
        service = np.random.choice([None,1,2,3],p=percent4)
        plating = np.random.choice([None,1,2,3],p=percent4)
        price = np.random.choice([None,1,2,3,4],p=percent5)        
        couple = np.random.choice([None,0],p=percent2)        
        family = np.random.choice([None,1],p=percent2)        
        solo = np.random.choice([None,2],p=percent2)        
        drink = np.random.choice([None,3],p=percent2)        
        friend = np.random.choice([None,4],p=percent2)        
        ##유저가 평가한 횟수 카운팅
        user_df.at[user_index,'count'] = user_df.iloc[user_index].loc['count'].tolist()+1
        id = user_df.iloc[user_index].loc['count']
        
        #db insert
        cursor.execute('''INSERT INTO young_cheline.evaluate (menu_id, time, user_name, cleaning, couple, drink, family, friend, number, 
                plating, price, restaurant_id, service, solo, flavor, url, id) VALUE(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
              ,(menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id))
        conn.commit()
        
        # 모두 엮어서 하나의 데이터프레임으로 만들기
        rws_data.append([menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id])
        
    result_df = pd.DataFrame(columns = ['menu_id','time','user_name','cleaning','couple','drink','family','friend','number','plating','price','restaurant_id','service','solo','flavor','url','id'],data = rws_data)
    return result_df

In [6]:
def get_mean_taste():
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute("select menu_id, restaurant_id,AVG(flavor) from young_cheline.evaluate group by menu_id, restaurant_id;")
    res = cursor.fetchall()
    return res

In [8]:
def get_count():
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute("select count(*) from young_cheline.evaluate group by menu_id;")
    g_menu_id = cursor.fetchall()

    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN couple LIKE '0' THEN 1 END) AS specific_count
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    couple = cursor.fetchall()
    
    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN family LIKE '1' THEN 1 END) AS specific_count
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    family = cursor.fetchall()

    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN solo LIKE '2' THEN 1 END) AS specific_count
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    solo = cursor.fetchall()

    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN drink LIKE '3' THEN 1 END) AS specific_count
                    FROM young_cheline.evaluate
                    GROUP BY menu_id;''')
    
    drink = cursor.fetchall()

    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN friend LIKE '4' THEN 1 END) AS specific_count
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    friend = cursor.fetchall()

    cursor.execute('''SELECT menu_id, AVG(NULLIF(cleaning, '')),AVG(NULLIF(plating, '')),AVG(NULLIF(price, '')),AVG(NULLIF(service, ''))
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    avg_exxtra = cursor.fetchall()

    cursor.execute('''SELECT menu_name FROM young_cheline.menu;''')
    menu_name = cursor.fetchall()

    eva = get_mean_taste()

    for menu_id in range(len(g_menu_id)):
        menu_count =g_menu_id[menu_id][0] #메뉴별 평가 갯수
        
        menu = eva[menu_id][0]
        restaurant_id = eva[menu_id][1]
        avg_flavor = eva[menu_id][2]
        avg_flavor = round(avg_flavor)
        in_menu_name = menu_name[menu_id][0]

        cleaning = avg_exxtra[menu_id][1]
        cleaning = round(cleaning) if isinstance(cleaning, float) else cleaning

        plating = avg_exxtra[menu_id][2]
        plating = round(cleaning) if isinstance(cleaning, float) else cleaning

        price = avg_exxtra[menu_id][3]
        price = round(cleaning) if isinstance(cleaning, float) else cleaning
        
        service = avg_exxtra[menu_id][4]
        service = round(cleaning) if isinstance(cleaning, float) else cleaning

        ## mood
        solo2 = 2 if  (solo[menu_id][1]/ menu_count) >0.8 else None
        couple0 = 0 if  (couple[menu_id][1]/ menu_count) >0.8 else None
        drink3 = 3 if  (drink[menu_id][1]/ menu_count) >0.8 else None
        family1 = 1 if  (family[menu_id][1]/ menu_count) >0.8 else None
        friend4 = 4 if  (friend[menu_id][1]/ menu_count) >0.8 else None

        cursor.execute('''INSERT INTO young_cheline.restaurant_evaluate
            (menu_id,menu_name, cleaning, plating, price, restaurant_id, service, flavor, couple, drink, family, friend, solo)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
                       (menu, in_menu_name,cleaning, plating, price, restaurant_id, service, avg_flavor, couple0, drink3, family1, friend4, solo2))

    
    conn.commit()

In [9]:
def get_data():
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute("SELECT flavor, menu_id, user_name FROM young_cheline.evaluate;")
    res = cursor.fetchall()
    return res

In [10]:
def make_ptable(df):
    ptable = df.pivot_table(values = 'flavor', index = 'menu_id', columns = 'user_name') 
    
    return ptable

In [11]:
def cosine(ptable):
    ptable = ptable.fillna(0)
    menu_id = ptable.index

    cos = cosine_similarity(ptable)
    cos = pd.DataFrame(data = cos, index=menu_id, columns=menu_id)
    return cos

In [12]:
def search_cos(cos,pt, user_id: str):
    ex = pt.assign(menu=pt.index.to_list())

    pt=ex.loc[ex[user_id].notnull(), [user_id, 'menu']] 
    user_cos=cos[pt['menu']] ## cos 확인 
    user_cos=user_cos[~user_cos.isin(user_cos.loc[pt['menu']])].dropna()
    X_vector = pt.iloc[:,[0]].values/10+1
    Y_matrix= user_cos.to_numpy()
    Matrix_product=Y_matrix.dot(X_vector)
    user_cos['Sum']=Matrix_product
    user_recomand=user_cos.sort_values('Sum',ascending=False).head(10).index.to_list()
    return user_recomand

In [13]:
def data_open():
    data = get_data()
    df = pd.DataFrame(data, columns=['flavor', 'menu_id','user_name'])
    df['flavor'] = df['flavor'].astype(int)
    return df

In [14]:
def recommand(user_id):
    df=data_open()
    ptable = make_ptable(df)
    cos = cosine(ptable)
    a = search_cos(cos,ptable, user_id=user_id)
    print(user_id,a)
    return user_id, a

In [15]:
def insert_db(list):
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()

    for a in list:
        menu_id = a[0]
        user_name = a[1]
        current_time = datetime.now()
        cursor.execute('''INSERT INTO young_cheline.recommend (menu_id,user_name,time)
                        VALUES (%s, %s,%s);''',(menu_id, user_name,current_time))
    conn.commit()